Посчитаем метрику Lifetime для данных о рейтингах фильмов. Т. е. среднее время "жизни" пользователя между первым и последним выставленным рейтингом. Нам необходимо выполнить следующие действия:

*Для каждого пользователя вычислить минимальное и максимальное значение timestamp
*Посчитать разницу минимального и максимального значения timestamp
*Усреднить полученную разницу для всех пользователейm

In [1]:
import pandas as pd
data_ratings = pd.read_csv('ratings.csv') 

movies_count = data_ratings.groupby('userId').count().reset_index() # получили датафрэйм где на каждый userId количество оценок
movies_count.head()


,userId,movieId,rating,timestamp
0,1,20,20,20
1,2,76,76,76
2,3,51,51,51
3,4,204,204,204
4,5,100,100,100


In [2]:
users_with_100_and_more = movies_count[ movies_count['rating'] >= 100 ] # получили датафрэйм с пользователями, поставившими более чем 100 оценок
users_with_100_and_more.head()

### РАБОТАЕТ ВЕРНО !!!

,userId,movieId,rating,timestamp
3,4,204,204,204
4,5,100,100,100
7,8,116,116,116
14,15,1700,1700,1700
16,17,363,363,363


In [3]:

list_of_users_with_100_and_more = users_with_100_and_more['userId'].tolist() # получили лист пользователей поставивших 100 и болле отметок
list_of_users_with_100_and_more[:6]

[4, 5, 8, 15, 17, 19]

In [4]:
data_clean = data_ratings[ data_ratings['userId'].isin( list_of_users_with_100_and_more ) ] # НЕТ !!!! НЕ получили датафрэйм с пользователями больше поставивших больше чем 100 отметок
data_clean.head()
#data_clean.groupby('userId').mean().reset_index().sort_values( by = 'rating', ascending = False ).head() - тут считаем среднее ,э то нам не надо

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


In [5]:
userLifetime = data_clean.groupby( 'userId' ).agg([min, max]).reset_index() # к предыдщему датафрэйму добавили расчет min и max
userLifetime.head()

userId movieId         rating        timestamp            
             min     max    min  max         min         max
0      4      10    4006    1.0  5.0   949778714   949982274
1      5       3   48385    1.5  5.0  1163373044  1163375145
2      8      32   44004    0.5  5.0  1154389340  1154474527
3     15       1  161155    0.5  5.0   997937239  1469330735
4     17       6   34437    0.5  5.0  1127468587  1127476640

In [6]:
"""Теперь считаем разницу в секундах первой и последней оценки:"""

userLifetime['diff'] = userLifetime['timestamp']['max'] - userLifetime['timestamp']['min']

userLifetime.head()

userId movieId         rating        timestamp                   diff
             min     max    min  max         min         max           
0      4      10    4006    1.0  5.0   949778714   949982274     203560
1      5       3   48385    1.5  5.0  1163373044  1163375145       2101
2      8      32   44004    0.5  5.0  1154389340  1154474527      85187
3     15       1  161155    0.5  5.0   997937239  1469330735  471393496
4     17       6   34437    0.5  5.0  1127468587  1127476640       8053

In [7]:
lifetime = userLifetime['diff'].mean()

#Если перевести результат в дни, то получим более наглядную цифру:
print('Lifetime в днях: {:.0f}'.format(lifetime / 3600 / 24))

Lifetime в днях: 455


In [8]:
### УРА ВСЕ ВЕРНО!!!